In [16]:
import os
import os.path as op

import seaborn as sns
from nimare.transforms import p_to_z
from nimare.results import MetaResult
from nilearn.image import threshold_img
from nilearn.plotting.cm import _cmap_d as nilearn_cmaps
import numpy as np
import matplotlib.image as mpimg
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
import gc
from nilearn.plotting import find_xyz_cut_coords
import os.path as op

import nibabel as nib
import numpy as np
from neuromaps.datasets import fetch_atlas
from nibabel.gifti import GiftiDataArray
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
from neuromaps import transforms
from nilearn.plotting import plot_stat_map
from nilearn import datasets
from matplotlib.gridspec import GridSpec
import numpy as np
from nimare.transforms import d_to_g, p_to_z, t_to_d, z_to_t
from neuromaps.datasets import fetch_fslr
from nilearn.plotting.cm import _cmap_d as nilearn_cmaps
from surfplot import Plot

In [2]:
CMAP = nilearn_cmaps["cold_hot"]

In [3]:
TASK_TO_HCP = {
    "working_memory_fmri_task_paradigm": "10_tfMRI_WM_2BK-0BK",
}

In [4]:
# Number of vertices in total without the medial wall
N_VERTICES = {
    "fsLR": {
        "32k": 59412,
        "164k": 298261,
    },
    "fsaverage": {
        "3k": 4661,
        "10k": 18715,
        "41k": 74947,
        "164k": 299881,
    },
    "civet": {
        "41k": 76910,
    },
}

# Number of vertices per hemisphere including the medial wall
N_VERTICES_PH = {
    "fsLR": {
        "32k": 32492,
        "164k": 163842,
    },
    "fsaverage": {
        "3k": 2562,
        "10k": 10242,
        "41k": 40962,
        "164k": 163842,
    },
    "civet": {
        "41k": 40962,
    },
}

In [5]:
def _rm_medial_wall(
    data_lh,
    data_rh,
    space="fsLR",
    density="32k",
    join=True,
    neuromaps_dir=None,
):
    """Remove medial wall from data in fsLR space.

    Data in 32k fs_LR space (e.g., Human Connectome Project data) often in
    GIFTI format include the medial wall in their data arrays, which results
    in a total of 64984 vertices across hemispheres. This function removes
    the medial wall vertices to produce a data array with the full 59412 vertices,
    which is used to perform functional decoding.

    This function was adapted from :func:`surfplot.utils.add_fslr_medial_wall`.

    Parameters
    ----------
    data : numpy.ndarray
        Surface vertices. Must have exactly 32492 vertices per hemisphere.
    join : bool
        Return left and right hemipsheres in the same arrays. Default: True.

    Returns
    -------
    numpy.ndarray
        Vertices with medial wall excluded (59412 vertices total).

    ValueError
    ------
    `data` has the incorrect number of vertices (59412 or 64984 only
        accepted)
    """
    assert data_lh.shape[0] == N_VERTICES_PH[space][density]
    assert data_rh.shape[0] == N_VERTICES_PH[space][density]

    atlas = fetch_atlas(space, density, data_dir=neuromaps_dir, verbose=0)

    medial_lh, medial_rh = atlas["medial"]
    wall_lh = nib.load(medial_lh).agg_data()
    wall_rh = nib.load(medial_rh).agg_data()

    data_lh = data_lh[np.where(wall_lh != 0)]
    data_rh = data_rh[np.where(wall_rh != 0)]

    if not join:
        return data_lh, data_rh

    data = np.hstack((data_lh, data_rh))
    assert data.shape[0] == N_VERTICES[space][density]
    return data


def _zero_medial_wall(data_lh, data_rh, space="fsLR", density="32k", neuromaps_dir=None):
    """Remove medial wall from data in fsLR space."""
    atlas = fetch_atlas(space, density, data_dir=neuromaps_dir, verbose=0)

    medial_lh, medial_rh = atlas["medial"]
    medial_arr_lh = nib.load(medial_lh).agg_data()
    medial_arr_rh = nib.load(medial_rh).agg_data()

    data_arr_lh = data_lh.agg_data()
    data_arr_rh = data_rh.agg_data()
    data_arr_lh[np.where(medial_arr_lh == 0)] = 0
    data_arr_rh[np.where(medial_arr_rh == 0)] = 0

    data_lh.remove_gifti_data_array(0)
    data_rh.remove_gifti_data_array(0)
    data_lh.add_gifti_data_array(GiftiDataArray(data_arr_lh))
    data_rh.add_gifti_data_array(GiftiDataArray(data_arr_rh))

    return data_lh, data_rh

In [6]:
def _vol_to_surf(metamap, space="fsLR", density="32k", return_hemis=False, neuromaps_dir=None):
    """Transform 4D metamaps from volume to surface space."""
    if space == "fsLR":
        metamap_lh, metamap_rh = transforms.mni152_to_fslr(metamap, fslr_density=density)
    elif space == "fsaverage":
        metamap_lh, metamap_rh = transforms.mni152_to_fsaverage(metamap, fsavg_density=density)
    elif space == "civet":
        metamap_lh, metamap_rh = transforms.mni152_to_civet(metamap, civet_density=density)

    metamap_lh, metamap_rh = _zero_medial_wall(
        metamap_lh,
        metamap_rh,
        space=space,
        density=density,
        neuromaps_dir=neuromaps_dir,
    )
    if return_hemis:
        return metamap_lh, metamap_rh

    metamap_arr_lh = metamap_lh.agg_data()
    metamap_arr_rh = metamap_rh.agg_data()

    metamap_surf = _rm_medial_wall(
        metamap_arr_lh,
        metamap_arr_rh,
        space=space,
        density=density,
        neuromaps_dir=neuromaps_dir,
    )

    return metamap_surf

In [7]:
def nifti_to_grayordinate(nifti_fn, axis):
    nifti = nib.load(nifti_fn)
    nifti_data = nifti.get_fdata(dtype=np.float32)

    n_go = axis.name.shape[0]
    grayordinate = np.zeros((n_go))

    volume_mask = axis.volume_mask
    vox_indices = tuple(axis.voxel[volume_mask].T)

    grayordinate[volume_mask] = nifti_data[vox_indices]
    grayordinate[~volume_mask] = _vol_to_surf(nifti_fn)
    
    return grayordinate

In [8]:
def plot_vol(nii_img_thr, threshold, out_file, mask_contours=None, coords=None, vmax=8, alpha=1, cmap=CMAP):
    template = datasets.load_mni152_template(resolution=1)

    display_modes = ["x", "y", "z"]
    fig = plt.figure(figsize=(5, 5))
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=None)
    gs = GridSpec(2, 2, figure=fig)

    for dsp_i, display_mode in enumerate(display_modes):
        if display_mode == "z":
            ax = fig.add_subplot(gs[:, 1], aspect="equal")
            colorbar = True
        else:
            ax = fig.add_subplot(gs[dsp_i, 0], aspect="equal")
            colorbar = False

        if coords is not None:
            cut_coords = [coords[dsp_i]]
            if np.isnan(cut_coords):
                cut_coords = 1
        else:
            cut_coords = 1

        display = plot_stat_map(
            nii_img_thr,
            bg_img=template,
            black_bg=False,
            draw_cross=False,
            annotate=True,
            alpha=alpha,
            cmap=cmap,
            threshold=threshold,
            symmetric_cbar=True,
            colorbar=colorbar,
            display_mode=display_mode,
            cut_coords=cut_coords,
            vmax=vmax,
            axes=ax,
        )
        if mask_contours:
            display.add_contours(mask_contours, levels=[0.5], colors="black")

    fig.savefig(out_file, bbox_inches="tight", dpi=300)


def plot_surf(map_lh, map_rh, out_file, mask_contours=None, vmax=8, cmap=CMAP):
    surfaces = fetch_fslr(density="32k")
    lh, rh = surfaces["inflated"]
    sulc_lh, sulc_rh = surfaces["sulc"]

    p = Plot(surf_lh=lh, surf_rh=rh, layout="grid")
    p.add_layer({"left": sulc_lh, "right": sulc_rh}, cmap="binary_r", cbar=False)
    p.add_layer(
        {"left": map_lh, "right": map_rh}, cmap=cmap, cbar=False, color_range=(-vmax, vmax)
    )
    if mask_contours:
        mask_lh, mask_rh = transforms.mni152_to_fslr(mask_contours, fslr_density="32k")
        mask_lh, mask_rh = _zero_medial_wall(
            mask_lh,
            mask_rh,
            space="fsLR",
            density="32k",
        )
        mask_arr_lh = mask_lh.agg_data()
        mask_arr_rh = mask_rh.agg_data()
        countours_lh = np.zeros_like(mask_arr_lh)
        countours_lh[mask_arr_lh != 0] = 1
        countours_rh = np.zeros_like(mask_arr_rh)
        countours_rh[mask_arr_rh != 0] = 1

        colors = [(0, 0, 0, 0)]
        contour_cmap = ListedColormap(colors, "regions", N=1)
        line_cmap = ListedColormap(["black"], "regions", N=1)
        p.add_layer(
            {"left": countours_lh, "right": countours_rh},
            cmap=line_cmap,
            as_outline=True,
            cbar=False,
        )
        p.add_layer(
            {"left": countours_lh, "right": countours_rh},
            cmap=contour_cmap,
            cbar=False,
        )
    fig = p.build()
    fig.savefig(out_file, bbox_inches="tight", dpi=300)

In [9]:
DPI = 300
VMAX = 12
C_EXTENT = 90
P_THRESH = 0.001
Z_THRESH = p_to_z(P_THRESH, tail="two")

CMAP = nilearn_cmaps["cold_hot"]

In [10]:
hcp_dir = "/Users/julioaperaza/Downloads/HCP_S1200_GroupAvg_v1"
hcp_task_dir = "/Users/julioaperaza/Documents/GitHub/large-scale-ibma/temp/hcp_tasks"
hcp_fn = op.join(hcp_dir, "HCP_S1200_997_tfMRI_ALLTASKS_level2_cohensd_hp200_s2_MSMAll.dscalar.nii")
hcp_maps = nib.load(hcp_fn)

In [11]:
hcp_data = hcp_maps.get_fdata(dtype=np.float32)
cifti_hdr = hcp_maps.header
hcp_axis = cifti_hdr.get_axis(1)

In [12]:
def _get_max_val(map_thr):
    data = map_thr.get_fdata()
    mask = ~np.isnan(data) & (data != 0)
    data = data[mask]
    max_val = 3 if len(data) == 0 else np.max(np.abs(data))
    return max_val

In [13]:
titles = [
    "Stouffers\nIndependent Contrasts",
    "Weighted Stouffers\nIndependent Contrasts",
    "Stouffers\nNon-independent Contrasts",
    "Weighted Stouffers\nNon-independent Contrasts",
    "Weighted Stouffers\nDownweighted by Study Size\nNon-independent Contrasts",
    "Fixed Effects\nHedges' g",
    "HCP",
]

In [ ]:
n_rows = 2
n_cols = 7
w = 1.5 * n_cols
h = 1.2 * n_rows

result_fns = [
    "no-hcp_stouffers_result.pkl.gz",
    "no-hcp_stouffers-weighted_result.pkl.gz",
    "no-hcp_stouffers-ncontrast_result.pkl.gz",
    "no-hcp_stouffers-ncontrast-weighted_result.pkl.gz",
    "no-hcp_stouffers-ncontrast-weighted-downweighted_result.pkl.gz",
    "no-hcp_fe-hedges_result.pkl.gz",
]

tasks = [
    # "emotion_processing_fmri_task_paradigm",
    # "language_processing_fmri_task_paradigm",
    # "motor_fmri_task_paradigm",
    # "social_cognition_(theory_of_mind)_fmri_task_paradigm",
    "working_memory_fmri_task_paradigm",
]
mode = "manual"

results_dir = '../results'
outputs_dir = './stouffers_variations'
outputs_dir = op.join(outputs_dir, mode)
os.makedirs(outputs_dir, exist_ok=True)
for task in tasks:
    out_file = op.join(outputs_dir, f"{task}.png")

    fig = plt.figure(figsize=(w, h))
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=None)
    gs = GridSpec(n_rows, n_cols, figure=fig)

    coords = None
    data_lst = []
    for col, result_fn in enumerate(result_fns):
        file_name = op.join(results_dir, "ibma", task, mode, result_fn)
        result = MetaResult.load(file_name)
        
        z_map = result.maps["z"]
        z_map = np.nan_to_num(z_map, nan=0.0)
        dof_map = result.maps["dof"]
        t_map = np.array([z_to_t(z_val, dof) for z_val, dof in zip(z_map, dof_map)])
        cohens_maps = t_to_d(t_map, dof_map + 1)
        hedges_maps = d_to_g(cohens_maps, dof_map + 1)

        # ibma_data = np.nan_to_num(ibma_data, nan=0.0)
        # ibma_img = nib.Nifti1Image(ibma_data, ibma_img.affine, ibma_img.header)
        ibma_img = result.masker.inverse_transform(hedges_maps)
        
        # Define temp out files
        ibma_img_fn = op.join(outputs_dir, f"{task}_map-{col}.nii.gz")
        ibma_img_thr_fn = op.join(outputs_dir, f"{task}_map-{col}_thr.nii.gz")
        nib.save(ibma_img, ibma_img_fn)

        # GEt image grayordinate for correlation matrix
        ibma_img_grayordinate = nifti_to_grayordinate(ibma_img_fn, hcp_axis)
        data_lst.append(ibma_img_grayordinate)

        # Plotting ===============================================================
        # Threshold image
        # img_thr = threshold_img(ibma_img, Z_THRESH, two_sided=True, cluster_threshold=C_EXTENT)
        if coords is None:
            # coords = find_xyz_cut_coords(img_thr)
            coords = find_xyz_cut_coords(ibma_img)

        
        # Get gifti for surface plotting
        # nib.save(img_thr, ibma_img_thr_fn)
        # img_thr_lh, img_thr_rh = _vol_to_surf(ibma_img_thr_fn, return_hemis=True)
        img_lh, img_rh = _vol_to_surf(ibma_img_fn, return_hemis=True)

        # Plotting
        vol_fn = op.join(outputs_dir, f"{task}_map-{col}_vol.png")
        surf_fn = op.join(outputs_dir, f"{task}_map-{col}_surf.png")
        # plot_vol(img_thr, Z_THRESH, vol_fn, coords=coords, vmax=VMAX)
        plot_vol(ibma_img, 0, vol_fn, coords=coords, vmax=1, cmap="coolwarm")
        plot_surf(img_lh, img_rh, surf_fn, vmax=1, cmap="coolwarm")

        # Add images to figure
        for row, img_file in enumerate([vol_fn, surf_fn]):
            img = mpimg.imread(img_file)
            ax = fig.add_subplot(gs[row, col], aspect="equal")
            ax.imshow(img)
            ax.set_axis_off()
        
            if row == 0:
                ax.set_title(titles[col], fontsize=5)

    # Plot HCP data
    hcp_img_lh = nib.load(op.join(hcp_task_dir, f"{TASK_TO_HCP[task]}_lh.gii"))
    hcp_img_rh = nib.load(op.join(hcp_task_dir, f"{TASK_TO_HCP[task]}_rh.gii")) 
    hcp_img = nib.load(op.join(hcp_task_dir, f"{TASK_TO_HCP[task]}_subcort.nii.gz"))
    hcp_img_grayordinate = np.load(op.join(hcp_task_dir, f"{TASK_TO_HCP[task]}.npy"))
    
    data_lst.append(hcp_img_grayordinate)
    vol_fn = op.join(outputs_dir, f"{task}_map-hcp_vol.png")
    surf_fn = op.join(outputs_dir, f"{task}_map-hcp_surf.png")
    plot_vol(hcp_img, 0, vol_fn, coords=coords, vmax=1, cmap="coolwarm")
    plot_surf(hcp_img_lh, hcp_img_rh, surf_fn, vmax=1, cmap="coolwarm")
    for row, img_file in enumerate([vol_fn, surf_fn]):
        img = mpimg.imread(img_file)
        ax = fig.add_subplot(gs[row, -1], aspect="equal")
        ax.imshow(img)
        ax.set_axis_off()
        
        if row == 0:
            ax.set_title(titles[-1], fontsize=5)
    
    # Add title
    fig.suptitle(task.replace("_", " "), fontsize=7)

    # Make sure the axis size if the same for different labels sizes
    fig.subplots_adjust(top=0.8)
    fig.savefig(out_file, bbox_inches="tight", dpi=DPI)
    fig.show()

    data_arr = np.array(data_lst)
    corr_mat = np.corrcoef(data_arr)

    fig_hm, ax_hm = plt.subplots(figsize=(10, 7))
    sns.heatmap(
        corr_mat, 
        annot=True, 
        fmt=".2f", 
        cmap="Reds",
        square=True,
        xticklabels=titles, 
        yticklabels=titles,
        ax=ax_hm,
    )
    fig_hm.savefig(op.join(outputs_dir, f"{task}_corr-mat.png"), bbox_inches="tight", dpi=DPI)
    plt.close()

In [ ]:
result_fn = op.join(results_dir, "hedges", "neurovault_result.pkl.gz")
result = MetaResult.load(result_fn)
img = result.get_map("z_corr-FDR_method-indep")

img_thr = threshold_img(img, Z_THRESH, two_sided=False, cluster_threshold=C_EXTENT)
nib.save(img_thr, op.join(outputs_dir, "hedges_thr.nii.gz"))

vol_fn = op.join(outputs_dir, "hedges_vol.png")
surf_fn = op.join(outputs_dir, "hedges_surf.png")

plot_vol(img_thr, Z_THRESH, vol_fn, vmax=VMAX)
plot_surf(op.join(outputs_dir, "hedges_thr.nii.gz"), surf_fn, vmax=VMAX)